<a href="https://colab.research.google.com/github/s-choung/CCEL_Multiscale/blob/main/bimetallic_surf_h_bind_data_generation_1205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is written by S.Choung for the purpose of generating HBE and OHBE of bimetallic alloy slabs, leveraging OCDATASET module, pymatgen, ase and OCP. For Jinuk. 5th Dec

## OCP installation

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
# Set the path to your pseudopotential files in Google Drive
pp_drive_path = '/content/drive/My Drive/potcars'  # Update this path to your actual directory
# Set the VASP_PP_PATH environment variable
os.environ['VASP_PP_PATH'] = pp_drive_path


Mounted at /content/drive


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .
!pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb
!pip install submitit

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'ocp'...
remote: Enumerating objects: 11841, done.
remote: Counting objects: 100% (2914/2914), done.
remote: Compressing objects: 100% (716/716), done.
remote: Total 11841 (delta 2506), reused 2351 (delta 2197), pack-reused 8927
Receiving objects: 100% (11841/11841), 172.64 MiB | 38.31 MiB/s, done.
Resolving deltas: 100% (8158/8158), done.
Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable fo

In [3]:
import json,zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm

from ase.io import read

from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from ocpmodels.datasets import SinglePointLmdbDataset
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import CGCNN,SchNet
from ocpmodels.datasets import data_list_collater
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
torch.cuda.is_available()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


False

In [4]:
!sudo apt-get install povray  ## visulization program

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-povray libsdl1.2debian povray-includes
Suggested packages:
  povray-doc povray-examples
The following NEW packages will be installed:
  fonts-povray libsdl1.2debian povray povray-includes
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,792 kB of archives.
After this operation, 5,199 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-povray all 1:3.7.0.10-1 [70.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libsdl1.2debian amd64 1.2.15+dfsg2-6 [177 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 povray amd64 1:3.7.0.10-1 [1,312 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 povray-includes all 1:3.7.0.10-1 [233 kB]
Fetched 1,792 kB in 1s (2,165 kB/s)
debconf: unable to initialize frontend: Dialo

In [5]:
from ase import Atoms
from ase.io import Trajectory, write
from ase.build import bulk, surface, molecule, add_adsorbate, fcc111
from ase.constraints import ExpCellFilter, StrainFilter, FixAtoms, FixedPlane, FixBondLength
from ase.optimize import LBFGS, BFGS, FIRE
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view
from ase.build.rotate import minimize_rotation_and_translation
from ase.md import MDLogger
from ase.io.vasp import read_vasp

import pandas as pd
import ipywidgets as widgets
from IPython.display import display_png, Image as ImageWidget
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import shutil

import glob
from pathlib import Path
from PIL import Image, ImageDraw

from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.vasp import Poscar
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor

import numpy as np
from scipy.spatial.distance import cdist
def visual(structure, index, max_size=(100, 100), stretch_y=1.0):
    renderer = write('./temp.pov', structure, rotation='15z,-90x')
    renderer.render()
    image_path = './temp.png'
    img = Image.open(image_path)

    # Calculate new size with stretch factor
    new_size = (max_size[0], int(max_size[1] * stretch_y))
    img = img.resize(new_size, Image.LANCZOS)
    display(img)
    # Move files to output directory
    files = ['./temp.ini', './temp.pov', './temp.png']
    destination = './output/'

    # Ensure destination directory exists
    os.makedirs(destination, exist_ok=True)

    for file in files:
        # Remove the file in the destination directory if it exists
        if os.path.isfile(os.path.join(destination, os.path.basename(file))):
            os.remove(os.path.join(destination, os.path.basename(file)))

        shutil.move(file, destination)


## load some useful utillities
 code taken from https://open-catalyst-project.github.io/tutorial/fine-tuning/fine-tuning-oxides.html

In [6]:
import ocpmodels as om
from pathlib import Path
import urllib
import os
from pathlib import Path
import requests
import subprocess
import sys
import numba
import numpy as np
import ase
import e3nn
import pymatgen.core as pc
import torch
import torch.cuda as tc
import torch_geometric as tg
import platform
import psutil

def ocp_root():
    """Return the root directory of the installed ocp package."""
    return Path(om.__file__).parent.parent

def ocp_main():
    """Return the path to ocp main.py"""
    return ocp_root() / "main.py"
def describe_ocp():
    """Print some system information that could be useful in debugging."""
    print(sys.executable, sys.version)
    print(f'ocp is installed at {ocp_root()}')

    commit_hash = (
            subprocess.check_output(
                    [
                        "git",
                        "-C",
                        om.__path__[0],
                        "describe",
                        "--always",
                    ]
                )
                .strip()
                .decode("ascii")
            )
    print(f'ocp repo is at git commit: {commit_hash}')
    print(f'numba: {numba.__version__}')
    print(f'numpy: {np.version.version}')
    print(f'ase: {ase.__version__}')
    print(f'e3nn: {e3nn.__version__}')
    print(f'pymatgen: {pc.__version__}')
    print(f'torch: {torch.version.__version__}')
    print(f'torch.version.cuda: {torch.version.cuda}')
    print(f'torch.cuda: is_available: {tc.is_available()}')
    if tc.is_available():
        print('  __CUDNN VERSION:', torch.backends.cudnn.version())
        print('  __Number CUDA Devices:', torch.cuda.device_count())
        print('  __CUDA Device Name:',torch.cuda.get_device_name(0))
        print('  __CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    print(f'torch geometric: {tg.__version__}')
    print()
    print(f'Platform: {platform.platform()}')
    print(f'  Processor: {platform.processor()}')
    print(f'  Virtual memory: {psutil.virtual_memory()}')
    print(f'  Swap memory: {psutil.swap_memory()}')
    print(f'  Disk usage: {psutil.disk_usage("/")}')


checkpoints = {
    # Open Catalyst 2020 (OC20)
    'CGCNN 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_200k.pt',
    'CGCNN 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_2M.pt',
    'CGCNN 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_20M.pt',
    'CGCNN All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_all.pt',
    'DimeNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_200k.pt',
    'DimeNet 2M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_2M.pt',
    'SchNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_200k.pt',
    'SchNet 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_2M.pt',
    'SchNet 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_20M.pt',
    'SchNet All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_all_large.pt',
    'DimeNet++ 200k'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_200k.pt',
    'DimeNet++ 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_2M.pt',
    'DimeNet++ 20M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_20M.pt',
    'DimeNet++ All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_all.pt',
    'SpinConv 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/spinconv_force_centric_2M.pt',
    'SpinConv All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/spinconv_force_centric_all.pt',
    'GemNet-dT 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/gemnet_t_direct_h512_2M.pt',
    'GemNet-dT All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt',
    'PaiNN All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_05/s2ef/painn_h512_s2ef_all.pt',
    'GemNet-OC 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_2M.pt',
    'GemNet-OC All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_all.pt',
    'GemNet-OC All+MD'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/gemnet_oc_base_s2ef_all_md.pt',
    'GemNet-OC-Large All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_large_s2ef_all_md.pt',
    'SCN 2M'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t1_b1_s2ef_2M.pt',
    'SCN-t4-b2 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t4_b2_s2ef_2M.pt',
    'SCN All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_all_md_s2ef.pt',
    'eSCN-L4-M2-Lay12 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l4_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_all_md_s2ef.pt',
    'eSCN-L6-M3-Lay20 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m3_lay20_all_md_s2ef.pt',
    'EquiformerV2 (83M) 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_83M_2M.pt',
    'EquiformerV2 (31M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_31M_ec4_allmd.pt',
    'EquiformerV2 (153M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_153M_ec4_allmd.pt',
    # Open Catalyst 2022 (OC22)
    'GemNet-dT OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gndt_oc22_all_s2ef.pt',
    'GemNet-OC OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt',
    'GemNet-OC OC20+OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC trained with `enforce_max_neighbors_strictly=False` #467 OC20+OC22' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_05/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC OC20->OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_finetune_all_s2ef.pt',
    'equiformer' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_10/oc22/s2ef/eq2_121M_e4_f100_oc22_s2ef.pt'
                }

def list_checkpoints():
    """List checkpoints that are available to download."""
    print('See https://github.com/Open-Catalyst-Project/ocp/blob/main/MODELS.md for more details.')
    for key in checkpoints:
        print(key)
    print('Copy one of these keys to get_checkpoint(key) to download it.')


def get_checkpoint(key):
    """Download a checkpoint.

    key: string in checkpoints.

    Returns name of checkpoint that was saved.
    """
    url = checkpoints.get(key, None)
    if url is None:
        raise Exception('No url found for {key}')

    pt = Path(urllib.parse.urlparse(url).path).name

    if not os.path.exists(pt):
        with open(pt, 'wb') as f:
            print(f'Downloading {url}')
            f.write(requests.get(url).content)
    return pt

In [7]:
# @title
from pathlib import Path
import numpy as np
from ase.db import connect

def train_test_val_split(ase_db, ttv=(0.8, 0.1, .1), files=('train.db', 'test.db', 'val.db'), seed=42):
    """Split an ase db into train, test and validation dbs.

    ase_db: path to an ase db containing all the data.
    ttv: a tuple containing the fraction of train, test and val data. This will be normalized.
    files: a tuple of filenames to write the splits into. An exception is raised if these exist.
           You should delete them first.
    seed: an integer for the random number generator seed

    Returns the absolute path to files.
    """

    for db in files:
        if os.path.exists(db):
            raise Exception('{db} exists. Please delete it before proceeding.')

    src = connect(ase_db)
    N = src.count()

    ttv = np.array(ttv)
    ttv /= ttv.sum()

    train_end = int(N * ttv[0])
    test_end = train_end + int(N * ttv[1])

    train = connect(files[0])
    test = connect(files[1])
    val = connect(files[2])

    ids = np.arange(1, N + 1)
    rng = np.random.default_rng(seed=42)
    rng.shuffle(ids)

    for _id in ids[0:train_end]:
        row = src.get(id=int(_id))
        train.write(row.toatoms())

    for _id in ids[train_end:test_end]:
        row = src.get(id=int(_id))
        test.write(row.toatoms())

    for _id in ids[test_end:]:
        row = src.get(id=int(_id))
        val.write(row.toatoms())

    return [Path(f).absolute() for f in files]

# @title
from yaml import load, dump
from yaml import CLoader as Loader, CDumper as Dumper
import torch
import os
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
from io import StringIO
import sys
import contextlib

def generate_yml_config(checkpoint_path, yml='run.yml', delete=(), update=()):
    """Generate a yml config file from an existing checkpoint file.

    checkpoint_path: string to path of an existing checkpoint
    yml: name of file to write to.
    pop: list of keys to remove from the config
    update: dictionary of key:values to update

    Use a dot notation in update.

    Returns an absolute path to the generated yml file.
    """

    # You can't just read in the checkpoint with torch. The calculator does some things to it.
    # Rather than recreate that here I just reuse the calculator machinery. I don't want to
    # see the output though, so I capture it.

    with contextlib.redirect_stdout(StringIO()) as _:
        config = OCPCalculator(checkpoint_path=checkpoint_path).config # there seems an parameter checkpoint changed in OCPcalculator (s.choung)

    for key in delete:
        if key in config and len(key.split('.')) == 1:
            del config[key]
        else:
            keys = key.split('.')
            if keys[0] in config:
                d = config[keys[0]]
            else:
                continue
            if isinstance(d, dict):
                for k in keys[1:]:
                    if isinstance(d[k], dict):
                        d = d[k]
                    else:
                        if k in d:
                            del d[k]

    def nested_set(dic, keys, value):
        for key in keys[:-1]:
            dic = dic.setdefault(key, {})
        dic[keys[-1]] = value

    for _key in update:
        keys = _key.split('.')
        nested_set(config, keys, update[_key])


    out = dump(config)
    with open(yml, 'wb') as f:
        f.write(out.encode('utf-8'))

    return Path(yml).absolute()

# OCDATA module installation

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
if not os.path.exists('tutorial'):
     !git clone https://github.com/Open-Catalyst-Project/tutorial.git
!git clone https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git

!pip install -e ./Open-Catalyst-Dataset
os.chdir('Open-Catalyst-Dataset')
!git clone -b OC22_dataset https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git


Cloning into 'tutorial'...
remote: Enumerating objects: 2147, done.
remote: Counting objects: 100% (1408/1408), done.
remote: Compressing objects: 100% (1134/1134), done.
remote: Total 2147 (delta 311), reused 1328 (delta 249), pack-reused 739
Receiving objects: 100% (2147/2147), 29.67 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (560/560), done.
Cloning into 'Open-Catalyst-Dataset'...
remote: Enumerating objects: 1822, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 1822 (delta 315), reused 340 (delta 284), pack-reused 1308
Receiving objects: 100% (1822/1822), 36.89 MiB | 22.53 MiB/s, done.
Resolving deltas: 100% (1137/1137), done.
Obtaining file:///content/ocp/Open-Catalyst-Dataset
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ocdata
Cloning into 'Open-Catalyst-Dataset'...
remote: Enumerating objects: 1822, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objec

example to use Bulk module.

'/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'
이 피클파일에는 아래 기준을 통과한 안정한 bulk구조들만 담겨있음.


Pymatgen (63) was used to search over all bulk materials in the Materials Project with nonpositive formation energies and energies above lower hulls of at most 0.1 eV/atom.
https://pubs.acs.org/doi/full/10.1021/acscatal.0c04525

bulk 모듈 사용예시 by GPT

In [9]:
import os
from ocdata.core.bulk import Bulk

# Path to the bulk database pickle file
bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'

# Initialize a Bulk object in various ways

# 1. Initialize a Bulk object by randomly selecting from the bulk database
bulk_random = Bulk(bulk_db_path=bulk_db_path)
print("Random Bulk:", bulk_random)

# 2. Initialize a Bulk object with a specific bulk ID from the database
bulk_id = 5  # Example bulk ID
bulk_with_id = Bulk(bulk_id_from_db=bulk_id, bulk_db_path=bulk_db_path)
print("Bulk with ID:", bulk_with_id)

# 3. Initialize a Bulk object with a specific source ID from the database
bulk_src_id = "mp-30"  # Example source ID
bulk_with_src_id = Bulk(bulk_src_id_from_db=bulk_src_id, bulk_db_path=bulk_db_path)
print("Bulk with Source ID:", bulk_with_src_id)

# Generate slabs from a Bulk object
max_miller_index = 1
slabs = bulk_random.get_slabs(max_miller=max_miller_index)
print(f"Number of slabs generated from random bulk: {len(slabs)}")

# Each slab can be further processed or analyzed as required
for i, slab in enumerate(slabs):
    print(f"Slab {i}: {slab}")


Random Bulk: Bulk: (IrNiTi2)
Bulk with ID: Bulk: (In)
Bulk with Source ID: Bulk: (Cu)
Number of slabs generated from random bulk: 7
Slab 0: Slab: (Ir12Ni12Ti24, (1, 1, 1), 0.04166666666666685, True)
Slab 1: Slab: (Ir12Ni12Ti24, (1, 1, 1), 0.12500000000000017, True)
Slab 2: Slab: (Ir12Ni12Ti24, (1, 1, 1), 0.04166666666666685, False)
Slab 3: Slab: (Ir12Ni12Ti24, (1, 1, 1), 0.12500000000000017, False)
Slab 4: Slab: (Ir16Ni16Ti32, (1, 1, 0), 0.12499999999999993, True)
Slab 5: Slab: (Ir16Ni16Ti32, (1, 0, 0), 0.125, True)
Slab 6: Slab: (Ir16Ni16Ti32, (1, 0, 0), 0.125, False)


get all bulks

In [10]:
import pickle
from ocdata.core.bulk import Bulk

bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'

# Load the entire bulk database
with open(bulk_db_path, 'rb') as file:
    bulk_db = pickle.load(file)

all_bulks = []
all_bulk_atoms_list=[]
for bulk_entry in bulk_db:
    # Using the atoms directly
    bulk_atoms = bulk_entry['atoms']
    bulk_obj = Bulk(bulk_atoms=bulk_atoms)
    all_bulks.append(bulk_obj)
    all_bulk_atoms_list.append(bulk_atoms)

print(len(all_bulk_atoms_list))
print(all_bulk_atoms_list[0].symbols)


11410
Re2


bimetallic 중에서 관심있는애들만 선별
screening criteria 수정가능. 일단3d-5d metal로만 고름, non toxic한애들만 고름

일단 bulk아톰수가 n개가 넘지 않는 단순한 bulk에 대해서만 컬렉팅하도록 짜놓긴했는데 이 n개라는 기준은 바뀌어야함.(n=8로 일단 짜놓음.)


In [30]:
from collections import Counter
from pymatgen.core.periodic_table import Element
import pickle
from ocdata.core.bulk import Bulk

# Define the screening criteria for elements
elements_to_do = [el.symbol for el in Element if el.number < 80 and 21 < el.number and 3 < el.group < 13]
exclude_elements = ['Pb', 'Tl', 'Hg', 'Cd', 'Mg', 'As']  # Elements to exclude
elements_to_do = [el for el in elements_to_do if el not in exclude_elements]
print(elements_to_do)
# Load the entire bulk database
bulk_db_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/bulks.pkl'
with open(bulk_db_path, 'rb') as file:
    bulk_db = pickle.load(file)

unary_bulk_info = {}
binary_bulk_info = {}

# Iterate over the bulk database to find unary and binary bulks
for idx, bulk_entry in enumerate(bulk_db):
    bulk_atoms = bulk_entry['atoms']
    elements = bulk_atoms.get_chemical_symbols()
    element_count = Counter(elements)

    # Check if the bulk is unary or binary, matches the criteria, and has 10 or fewer atoms
    if all(el in elements_to_do for el in element_count) and len(bulk_atoms) <= 8:
        bulk_chemical_formula = bulk_atoms.get_chemical_formula()
        if len(element_count) == 1:
            unary_bulk_info[idx] = {'src_id': bulk_entry.get('src_id', None), 'formula': bulk_chemical_formula}
        elif len(element_count) == 2:
            binary_bulk_info[idx] = {'src_id': bulk_entry.get('src_id', None), 'formula': bulk_chemical_formula}

print("Unary bulk info:", unary_bulk_info)
print("Binary bulk info:", binary_bulk_info)
print("Unary bulk info:", len(unary_bulk_info))
print("Binary bulk info:", len(binary_bulk_info))

['Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au']
Unary bulk info: {0: {'src_id': 'mp-8', 'formula': 'Re2'}, 2: {'src_id': 'mp-49', 'formula': 'Os2'}, 3: {'src_id': 'mp-91', 'formula': 'W'}, 6: {'src_id': 'mp-146', 'formula': 'V'}, 7: {'src_id': 'mp-101', 'formula': 'Ir'}, 9: {'src_id': 'mp-74', 'formula': 'Rh'}, 10: {'src_id': 'mp-54', 'formula': 'Co2'}, 11: {'src_id': 'mp-13', 'formula': 'Fe'}, 12: {'src_id': 'mp-30', 'formula': 'Cu'}, 14: {'src_id': 'mp-81', 'formula': 'Au'}, 17: {'src_id': 'mp-124', 'formula': 'Ag'}, 18: {'src_id': 'mp-2', 'formula': 'Pd'}, 19: {'src_id': 'mp-126', 'formula': 'Pt'}, 20: {'src_id': 'mp-23', 'formula': 'Ni'}, 21: {'src_id': 'mp-79', 'formula': 'Zn2'}, 22: {'src_id': 'mp-33', 'formula': 'Ru2'}, 25: {'src_id': 'mp-129', 'formula': 'Mo'}, 26: {'src_id': 'mp-90', 'formula': 'Cr'}, 28: {'src_id': 'mp-72', 'formula': 'Ti3'}, 29: {'src_id': 'mp-1008634', 'form

for loop로 Slab 짜는 코드. 일단 111만 만들도록 짜놓음. 잘돌아가는지까지만 확인함. 여러 termination이 있는경우엔.. 어떻게 걸러낼지 모르겠당.

In [50]:
from ocdata.core.bulk import Bulk
from ocdata.core.slab import Slab

def generate_slabs_for_bulk(bulk_info_dict, bulk_db_path, max_miller_index, num_entries=None):
    all_slabs = {}

    # Generate slabs for the first 10 binary bulks # 이거 일단 10개 테스트용으로 짠거라 num entries를 없애면됨.
    items_to_process = list(bulk_info_dict.items())[:num_entries]

    for bulk_id, bulk_info in items_to_process:
        bulk_src_id = bulk_info['src_id']
        # Initialize the Bulk object
        if bulk_src_id is not None:
            bulk_obj = Bulk(bulk_src_id_from_db=bulk_src_id, bulk_db_path=bulk_db_path)
        else:
            bulk_obj = Bulk(bulk_id_from_db=bulk_id, bulk_db_path=bulk_db_path)

        specific_millers = (1, 1, 1)
        slabs = Slab.from_bulk_get_specific_millers(specific_millers, bulk=bulk_obj, min_ab=5.0)
        all_slabs[bulk_id] = (slabs[0].atoms, bulk_obj)  # atoms형태로 이렇게 바꿀수있음. test용으로 first_10_binary_bulk_slabs이걸로 돌려보는겨 일단 여러 termaination slab중에서 첫번째꺼만 해봄.
        print(f"Number of slabs generated from bulk ID {bulk_id} ({bulk_info['formula']}): {len(slabs)}")# len slabs가 termination 갯수임. 이중에서 첫번째꺼만 가져온거임.all_slabs[bulk_id] = slabs[0].atoms
        print(slabs[0].atoms)  # Print the first slab atoms

    return all_slabs

max_miller_index = 1

# Generate slabs for the first 10 binary bulks # 이거 일단 10개 테스트용으로 짠거라 num entries를 없애면됨.
print("Generating slabs for first 10 binary bulks...")
first_10_binary_bulk_slabs = generate_slabs_for_bulk(binary_bulk_info, bulk_db_path, max_miller_index, num_entries=10)


Generating slabs for first 10 binary bulks...
Number of slabs generated from bulk ID 239 (Pt2V): 3
Atoms(symbols='V4Pt8V4Pt8V4Pt8V4Pt8', pbc=True, cell=[[8.940824000508993, 0.0, -3.0315669752663474], [2.2352060001272482, 8.454731993600122, 1.602308620209781], [0.0, 0.0, 33.04280509636915]], bulk_equivalent=..., bulk_wyckoff=..., tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 44, 47]))
Number of slabs generated from bulk ID 240 (Pd2V): 3
Atoms(symbols='V4Pd8V4Pd8V4Pd8V4Pd8', pbc=True, cell=[[8.94696842535961, 0.0, -2.987064418591685], [2.236742106339903, 8.36907483597483, 1.6113422188446385], [0.0, 0.0, 33.01351652889585]], bulk_equivalent=..., bulk_wyckoff=..., tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 44, 47]))
Number of slabs generated from bulk ID 244 (OsV): 2
A

ready adsorbate OH and H

In [54]:
for bulk_id, (slab_atoms, corresponding_bulk) in first_10_binary_bulk_slabs.items():
  print(bulk_id)
  print(slab_atoms)
  print(corresponding_bulk)

239
Atoms(symbols='V4Pt8V4Pt8V4Pt8V4Pt8', pbc=True, cell=[[8.940824000508993, 0.0, -3.0315669752663474], [2.2352060001272482, 8.454731993600122, 1.602308620209781], [0.0, 0.0, 33.04280509636915]], bulk_equivalent=..., bulk_wyckoff=..., tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 44, 47]))
Bulk: (Pt2V)
240
Atoms(symbols='V4Pd8V4Pd8V4Pd8V4Pd8', pbc=True, cell=[[8.94696842535961, 0.0, -2.987064418591685], [2.236742106339903, 8.36907483597483, 1.6113422188446385], [0.0, 0.0, 33.01351652889585]], bulk_equivalent=..., bulk_wyckoff=..., tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 44, 47]))
Bulk: (Pd2V)
244
Atoms(symbols='V3Os3V3Os3V3Os3V3Os3V3Os3V3Os3V3Os3V3Os3', pbc=True, cell=[[8.555799606175656, 0.0, 5.238916300924601e-16], [4.277899803087827, 7.409539808637016, 5.238

In [52]:
import pickle
from ocdata.core import Adsorbate, Slab
from ocdata.core.adsorbate_slab_config import AdsorbateSlabConfig

# Load the hydrogen adsorbate from the adsorbate database
adsorbates_pkl_path = '/content/ocp/Open-Catalyst-Dataset/ocdata/databases/pkls/adsorbates.pkl'
with open(adsorbates_pkl_path, 'rb') as file:
    adsorbates_db = pickle.load(file)
print(adsorbates_db)
hydrogen_adsorbate = Adsorbate(adsorbate_id_from_db=1, adsorbate_db_path=adsorbates_pkl_path)

# Assuming first_10_binary_bulk_slabs contains both slab atoms and bulk objects
binary_bulk_adslabs = {}

for bulk_id, (slab_atoms, corresponding_bulk) in first_10_binary_bulk_slabs.items():
    adslabs = []
    # Create AdsorbateSlabConfig for the slab
    adslab_config = AdsorbateSlabConfig(
        slab=Slab(bulk=corresponding_bulk, slab_atoms=slab_atoms),
        adsorbate=hydrogen_adsorbate,
        mode="heuristic", #num_sites=1,  # Number of sites to sample (can be adjusted)
        num_augmentations_per_site=1,  # Number of augmentations per site
        interstitial_gap=0.1  # Gap between adsorbate and slab
    )
    adslabs.extend(adslab_config.atoms_list)

    binary_bulk_adslabs[bulk_id] = adslabs
    print(f"Generated {len(adslabs)} adslabs for bulk ID {bulk_id}")


{0: (Atoms(symbols='O', pbc=False), '*O', array([0]), '-H<sub>2(g)</sub> + H<sub>2</sub>O<sub>(g)</sub> + * &rarr; *O'), 1: (Atoms(symbols='H', pbc=False), '*H', array([0]), '0.5 H<sub>2(g)</sub> + * &rarr; *H'), 2: (Atoms(symbols='OH', pbc=False), '*OH', array([0]), '- -0.5 H<sub>2(g)</sub> + H<sub>2</sub>O<sub>(g)</sub> + * &rarr; *OH'), 3: (Atoms(symbols='H2O', pbc=False), '*OH2', array([2]), 'H<sub>2</sub>O<sub>(g)</sub> + * &rarr; *OH2'), 4: (Atoms(symbols='C', pbc=False), '*C', array([0]), 'CO<sub>(g)</sub> + H<sub>2(g)</sub> - H<sub>2</sub>O<sub>(g)</sub> + * &rarr; *C'), 5: (Atoms(symbols='CO', pbc=False), '*CO', array([0]), 'CO<sub>(g)</sub> + * &rarr; *CO'), 6: (Atoms(symbols='CH', pbc=False), '*CH', array([0]), 'CO<sub>(g)</sub> + 1.5 H<sub>2(g)</sub> - H<sub>2</sub>O<sub>(g)</sub> + * &rarr; *CH'), 7: (Atoms(symbols='CHO', pbc=False), '*CHO', array([0]), 'CO<sub>(g)</sub> + 0.5 H<sub>2(g)</sub> + * &rarr; *CHO'), 8: (Atoms(symbols='CHO', pbc=False), '*COH', array([0]), 'CO<

위처럼 H붙인 slab만드는거까지는 gpt랑 같이짜거, 위까지 짜는데 1시간내외정도 걸림.문제는 adslab갯수가 장난아니라서 이걸 어떻게 쳐내야될지 기준을 수립하긴해야될듯. heuristicmethod말고 random method로 site 5개만 조사한다 뭐 이런기준도 괜찮을것같고.일단 12/20미팅용으로는 bulk수를 줄이든 ,111-like만 조사하든, random site 5개만 하든 해서, gemnet계산해야되는 slab수를 최대한 줄이자. 위와같은 방식으로 OH 흡착형태도 만들면된데이.